In [2]:
# Standard library imports
import argparse
import json
import os
import subprocess
import sys
import time
from timeit import default_timer as timer
import logging
# Third party imports
import pennylane as qml
from pennylane import numpy as np
import jax.numpy as jnp
from pennylane import qjit 
from qiskit import QuantumCircuit


In [ ]:
import jax
jax.devices()

In [ ]:
%%time

for n_wires in [29, 30, 31]:
    n_layers = 2
    rank = 0
    num_runs = 1
    dev = qml.device("lightning.qubit", wires=n_wires)

    # Create QNode of device and circuit
    def circuit_adj(weights):
        qml.StronglyEntanglingLayers(weights, wires=list(range(n_wires)))
        return [qml.expval(qml.PauliZ(i)) for i in range(n_wires)]
        #return qml.expval(qml.PauliZ(0))

    params = jnp.array(
        np.random.random(qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_wires)), 
        dtype=jnp.float64
    )

    enable_jacobian = False
    if enable_jacobian:
        diff_method = "adjoint"
        # print(f"Initializing QNode with jacobian enabled: interface=autograd, diff_method={diff_method}")
        circuit_adj = qml.qnode(dev, interface="autograd", diff_method=diff_method)(circuit_adj)
    else:
        # print("Initializing QNode without jacobian")
        circuit_adj = qml.qnode(dev)(circuit_adj) 

    enable_qjit = True
    if enable_qjit:
        circuit_adj = qjit(circuit_adj)

    # Create MetricsWriter instance if rank 0
    timing = []
    for t in range(num_runs):
        start = time.time()            
        if enable_jacobian:
            # print("Calculating Jacobian")
            result = qml.jacobian(circuit_adj)(params)
        else:
            # print("Calculating State Vector without Jacobian")
            result = circuit_adj(params)
        end = time.time()
        timing.append(end - start)

    # Calculate and print average time
    avg_time = np.mean(timing)
    print(f"Number Qubits: {n_wires}, Number Layers: {n_layers}, Device: {dev.name}, JIT: {enable_qjit}, Jacobian: {enable_jacobian}, Average time per run: {avg_time:.4f} seconds")

In [ ]:
# Export the circuit to OpenQASM
qiskit_dev = qml.device("qiskit.aer", wires=n_wires)

@qml.qnode(qiskit_dev)
def circuit_qiskit(weights):
    qml.StronglyEntanglingLayers(weights, wires=list(range(n_wires)))
    return qml.math.hstack([qml.expval(qml.PauliZ(i)) for i in range(n_wires)])

circuit_qiskit(params)

qiskit_circuit = qiskit_dev._circuit  # Access the Qiskit QuantumCircuit object


In [ ]:
qasm_code = qiskit_circuit.qasm()  # Export to OpenQASM

# Save the QASM code to a file
with open("circuit.qasm", "w") as f:
    f.write(qasm_code)

# Print the QASM code
print(qasm_code)

In [ ]:
dev.name